In [1]:
import torch
import sigkernel2
import sigkernel
import csv
import timeit
import math
device = torch.cuda.device('cuda')

In [2]:
def generate(batch_size, length, dimension, device = torch.device('cpu')):
  random_walks = torch.randn(batch_size, length, dimension, dtype = torch.double, device = device)
  random_walks = torch.cumsum(random_walks, dim=1)
  return random_walks

In [4]:
def bench_memory(func):
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    func()
    return torch.cuda.max_memory_allocated() / 1024 ** 2

In [15]:
d_order = 0

l = 600
b = 100

torch.cuda.empty_cache()
sig = sigkernel2.SigKernel(sigkernel2.RBFKernel(1), dyadic_order = d_order)
sig1 = sigkernel.SigKernel(sigkernel.RBFKernel(1), dyadic_order = d_order)

x = generate(b, l, 7, device = torch.device('cuda:0'))

print(bench_memory(lambda: sig.compute_kernel(x, x, lean=True, max_batch=500)))
print(bench_memory(lambda: sig.compute_kernel(x, x, lean=False, max_batch=500)))
print(bench_memory(lambda: sig1.compute_kernel(x, x, max_batch=500)))

828.09521484375
828.09521484375
1373.35498046875


/home/paperspace/.local/lib/python3.9/site-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 100 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
